# Dev: Body TrackingのCSVデータを、 Unity用に変換

In [3]:
from pathlib import Path

import numpy as np
import pandas as pd

### const

In [4]:
KEY_TRANSLATOR_3DPOSE_TO_K4ABT = {
    0: {
        "name": "尻(尾てい骨)",
        "k4a_ID": 0,
        "k4a_name": "PELVIS",
    },
    1: {
        "name": "右尻(右足付け根)",
        "k4a_ID": 22,
        "k4a_name": "HIP_RIGHT",
    },
    2: {
        "name": "右ひざ",
        "k4a_ID": 23,
        "k4a_name": "KNEE_RIGHT",
    },
    3: {
        "name": "右足首",
        "k4a_ID": 24,
        "k4a_name": "ANKLE_RIGHT",
    },
    4: {
        "name": "左尻(左足付け根)",
        "k4a_ID": 18,
        "k4a_name": "HIP_LEFT",
    },
    5: {
        "name": "左ひざ",
        "k4a_ID": 19,
        "k4a_name": "KNEE_LEFT",
    },
    6: {
        "name": "左足首",
        "k4a_ID": 20,
        "k4a_name": "ANKLE_LEFT",
    },
    7: {
        "name": "脊椎",
        "k4a_ID": 1,
        "k4a_name": "SPINE_NAVAL",
    },
    8: {
        "name": "胸",
        "k4a_ID": 2,
        "k4a_name": "SPINE_CHEST",
    },
    9: {
        "name": "首/鼻",
        "k4a_ID": 3,
        "k4a_name": "NECK", 
    },
    10: {
        "name": "頭",
        "k4a_ID": 26,
        "k4a_name": "HEAD",
    },
    11: {
        "name": "左肩",
        "k4a_ID": 5,
        "k4a_name": "SHOULDER_LEFT",
    },
    12: {
        "name": "左ひじ",
        "k4a_ID": 6,
        "k4a_name": "ELBOW_LEFT",
    },
    13: {
        "name": "左手首",
        "k4a_ID": 7,
        "k4a_name": "WRIST_LEFT",
    },
    14: {
        "name": "右肩", 
        "k4a_ID": 12,
        "k4a_name": "SHOULDER_RIGHT",
    },
    15: {
        "name": "右ひじ",
        "k4a_ID": 13,
        "k4a_name": "ELBOW_RIGHT",
    },
    16: {
        "name": "右手首",
        "k4a_ID": 14,
        "k4a_name": "WRIST_RIGHT",
    },
}

In [5]:
KEY_TRANSLATOR_3DPOSE_TO_K4ABT 

{0: {'name': '尻(尾てい骨)', 'k4a_ID': 0, 'k4a_name': 'PELVIS'},
 1: {'name': '右尻(右足付け根)', 'k4a_ID': 22, 'k4a_name': 'HIP_RIGHT'},
 2: {'name': '右ひざ', 'k4a_ID': 23, 'k4a_name': 'KNEE_RIGHT'},
 3: {'name': '右足首', 'k4a_ID': 24, 'k4a_name': 'ANKLE_RIGHT'},
 4: {'name': '左尻(左足付け根)', 'k4a_ID': 18, 'k4a_name': 'HIP_LEFT'},
 5: {'name': '左ひざ', 'k4a_ID': 19, 'k4a_name': 'KNEE_LEFT'},
 6: {'name': '左足首', 'k4a_ID': 20, 'k4a_name': 'ANKLE_LEFT'},
 7: {'name': '脊椎', 'k4a_ID': 1, 'k4a_name': 'SPINE_NAVAL'},
 8: {'name': '胸', 'k4a_ID': 2, 'k4a_name': 'SPINE_CHEST'},
 9: {'name': '首/鼻', 'k4a_ID': 3, 'k4a_name': 'NECK'},
 10: {'name': '頭', 'k4a_ID': 26, 'k4a_name': 'HEAD'},
 11: {'name': '左肩', 'k4a_ID': 5, 'k4a_name': 'SHOULDER_LEFT'},
 12: {'name': '左ひじ', 'k4a_ID': 6, 'k4a_name': 'ELBOW_LEFT'},
 13: {'name': '左手首', 'k4a_ID': 7, 'k4a_name': 'WRIST_LEFT'},
 14: {'name': '右肩', 'k4a_ID': 12, 'k4a_name': 'SHOULDER_RIGHT'},
 15: {'name': '右ひじ', 'k4a_ID': 13, 'k4a_name': 'ELBOW_RIGHT'},
 16: {'name': '右手首', 'k4a

### functions

In [6]:
def to_unity_format(s):
    """Get a single row and convert it into unity format.
    Args:
        s (pd.Series): -
    Return:
        str
    """
    
    line = ""
    for joint_id in range(17):
        d = KEY_TRANSLATOR_3DPOSE_TO_K4ABT.get(joint_id)
        
        k4a_joint_id = d.get("k4a_ID")        
        px, py, pz =  s[f"J{k4a_joint_id:0=2}_P0"], s[f"J{k4a_joint_id:0=2}_P1"], s[f"J{k4a_joint_id:0=2}_P2"]
        
        line += f"{joint_id} {px} {pz} {py}, "
    return line

In [15]:
def select_body_index(df, index=0):
    """
    Note:
        複数のBodyが検出されているケースは、ここで消すべきだろうか？
    """
    df = df[df["body_index"] == 0].reset_index(drop=True)
    
    return df
    

def resample_to_15hz(df):
    df_new = df.copy()

    df["timestamp"] = pd.to_datetime(df["timestamp"], unit='s')
    df = df.set_index("timestamp").asfreq("66ms", method="ffill")
    
    return df.reset_index(drop=False)

In [16]:
def write_pos_txt(df, fname):    
    with open(fname, 'w', encoding='utf-8', newline='\n') as f:
        for i in range(1000, 5000):
            line = to_unity_format(df.loc[i, :])
            f.write(line + '\n')
    return True

### Conversion

In [17]:
input_file = "./data/U0002_S0100.csv"

# Load File
df = pd.read_csv(input_file)
# display(df.head())

# Timestamp関連
df = select_body_index(df, index=0)
df = resample_to_15hz(df)

,timestamp,body_index,J00_P0,J00_P1,J00_P2,J00_O0,J00_O1,J00_O2,J00_O3,J00_CONF,...,J30_O3,J30_CONF,J31_P0,J31_P1,J31_P2,J31_O0,J31_O1,J31_O2,J31_O3,J31_CONF
0,1.634869e+09,0,47.8702,348.741,2230.89,0.550014,-0.434476,0.474469,-0.532536,2,...,0.162899,2,-18.7838,-187.917,2152.40,0.630032,-0.257972,0.613827,-0.399658,2
1,1.634869e+09,0,46.8710,351.886,2230.96,0.549369,-0.436875,0.473526,-0.532078,2,...,0.193177,2,-15.2510,-198.904,2153.60,0.653550,-0.291262,0.585977,-0.380356,2
2,1.634869e+09,0,47.0129,353.611,2231.05,0.549459,-0.437178,0.473282,-0.531952,2,...,0.222863,2,-8.1355,-215.948,2156.89,0.665434,-0.370231,0.545385,-0.350259,2
3,1.634869e+09,0,42.4397,363.462,2242.18,0.572130,-0.413092,0.451773,-0.545823,2,...,0.241729,2,-12.7205,-215.574,2146.01,0.657378,-0.317591,0.606166,-0.315521,2
4,1.634869e+09,0,44.0698,365.538,2232.18,0.536085,-0.453433,0.475391,-0.530108,2,...,0.253010,2,-8.6541,-226.098,2153.21,0.675698,-0.352522,0.564011,-0.317887,2


In [18]:
df.head()

,timestamp,body_index,J00_P0,J00_P1,J00_P2,J00_O0,J00_O1,J00_O2,J00_O3,J00_CONF,...,J30_O3,J30_CONF,J31_P0,J31_P1,J31_P2,J31_O0,J31_O1,J31_O2,J31_O3,J31_CONF
0,2021-10-22 02:15:46.000406016,0,47.8702,348.741,2230.89,0.550014,-0.434476,0.474469,-0.532536,2,...,0.162899,2,-18.7838,-187.917,2152.40,0.630032,-0.257972,0.613827,-0.399658,2
1,2021-10-22 02:15:46.066406016,0,47.8702,348.741,2230.89,0.550014,-0.434476,0.474469,-0.532536,2,...,0.162899,2,-18.7838,-187.917,2152.40,0.630032,-0.257972,0.613827,-0.399658,2
2,2021-10-22 02:15:46.132406016,0,46.8710,351.886,2230.96,0.549369,-0.436875,0.473526,-0.532078,2,...,0.193177,2,-15.2510,-198.904,2153.60,0.653550,-0.291262,0.585977,-0.380356,2
3,2021-10-22 02:15:46.198406016,0,47.0129,353.611,2231.05,0.549459,-0.437178,0.473282,-0.531952,2,...,0.222863,2,-8.1355,-215.948,2156.89,0.665434,-0.370231,0.545385,-0.350259,2
4,2021-10-22 02:15:46.264406016,0,42.4397,363.462,2242.18,0.572130,-0.413092,0.451773,-0.545823,2,...,0.241729,2,-12.7205,-215.574,2146.01,0.657378,-0.317591,0.606166,-0.315521,2


In [9]:
df.to_csv("./data/U0002_S0100_cleaned.csv", index=False)